In [7]:
import datetime
import json
import os
import pandas as pd
import requests
import sqlalchemy

In [8]:
SRC_PATH = os.environ.get('SRC_PATH')  #  path to source code os.path.abspath('.')
DB_NAME = os.environ.get('DB_NAME')  #  [database will be one directory above source code]
IEX_KEY = os.environ.get('IEX_KEY')  # IEX api key to get market data

# Create engine
engine = sqlalchemy.create_engine(f"sqlite:///{SRC_PATH.replace('/MarketView', '')}/{DB_NAME}")

In [9]:
pd.read_csv('quote_dev.csv').set_index('symbol').to_sql('quote_dev', engine) # store development data in database

In [10]:
last_table_name = sorted(list(filter(lambda table_name: 'last' in table_name, engine.table_names())), reverse=True)[0]
pd.read_sql(last_table_name, engine, index_col='symbol')

/home/ec2-user/lab/lab_venv/lib64/python3.7/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


,price,size,time
symbol,,,
GBF,109.990,55,1659110492363
WBK,14.220,200,1643403588574
GBGR,16.165,1,1658250932698
PBA,37.860,49,1659125603299
SPAK,14.050,127,1659120682369
...,...,...,...
TEQI,34.460,4,1659117354875
BAC-K,25.380,76,1659124759081
TUGC,9.860,85,1651262398313


In [11]:
pd.read_sql('quote_dev', engine, index_col='symbol')

,sector,securityType,bidPrice,bidSize,askPrice,askSize,lastUpdated,lastSalePrice,lastSaleSize,lastSaleTime,volume
symbol,,,,,,,,,,,
BOAC+,None,None,0.00,0,0.00,0,1659101400000,0.000,0,0,0
LVTX,None,None,2.40,100,2.93,100,1659103527822,2.590,100,1659103160683,100
WIA,miscellaneous,cef,9.47,100,11.19,100,1659105711543,0.000,0,0,0
PCG-D,utilities,ps,0.00,0,0.00,0,1659101400000,0.000,0,0,0
CVLG,transportation,cs,29.98,133,33.50,478,1659110386836,32.690,100,1659110386836,2245
...,...,...,...,...,...,...,...,...,...,...,...
KEY-J,finance,ps,22.86,100,26.98,100,1659105976095,24.910,100,1659109737443,828
UI,electronictechnology,cs,272.29,100,325.71,100,1659109847064,299.060,5,1659108357335,303
LCF,None,None,23.30,100,27.41,100,1659107063353,0.000,0,0,0


In [16]:
combined_df = pd.read_sql(last_table_name, engine, index_col='symbol').merge(
    pd.read_sql('quote_dev', engine, index_col='symbol'),
    on='symbol'
)

combined_df

,price,size,time,sector,securityType,bidPrice,bidSize,askPrice,askSize,lastUpdated,lastSalePrice,lastSaleSize,lastSaleTime,volume
symbol,,,,,,,,,,,,,,
GBF,109.990,55,1659110492363,miscellaneous,et,0.00,0,0.00,0,1659101400000,0.000,0,0,0
GBGR,16.165,1,1658250932698,miscellaneous,et,0.00,0,0.00,0,1659101400000,0.000,0,0,0
PBA,37.860,49,1659125603299,industrialservices,cs,38.16,100,38.18,100,1659110420807,38.185,10,1659110356734,5695
SPAK,14.050,127,1659120682369,miscellaneous,et,12.82,100,15.08,100,1659106794453,0.000,0,0,0
PGC,32.700,9,1659124784767,finance,cs,32.46,233,32.60,200,1659110410236,32.470,100,1659109802688,463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XDJA,19.060,100,1655224696055,None,None,21.05,100,0.00,0,1659110234794,0.000,0,0,0
SARK,54.760,5,1659124797931,None,None,51.12,100,55.76,100,1659110420807,55.750,100,1659110137071,5170
TEQI,34.460,4,1659117354875,miscellaneous,et,31.50,100,37.42,100,1659102807082,0.000,0,0,0


In [41]:
def prepare_df(combined_df):
    prepared_df = (
        combined_df
        
        
        .assign(
            prev_close = lambda df: df['price'],
            last = lambda df: df['lastSalePrice'],
        )
        
        
        .drop(columns=[
            'price',
            'bidSize',
            'askSize',
            'size',
            'time',
            'sector',
            'securityType',
            'lastUpdated',
            'lastSaleSize',
            'lastSaleTime',
            'volume',
            ]
        )
        
    )
    
    return prepared_df

In [42]:
prepared_df = prepare_df(combined_df)

In [47]:
def add_features(prepared_df):
    featured_df = (
        
        prepared_df[
            (prepared_df['bidPrice'] > 0)
            & (prepared_df['askPrice'] > 0)
            & (prepared_df['last'] > 0)
        ]
    
        
        .assign(
            spd_pct = lambda df: (df['askPrice'] / df['bidPrice'] - 1) * 100,
            pct_chg = lambda df: (df['last'] / df['prev_close'] - 1) * 100,
        )
    
        
    )
    
    return featured_df

In [48]:
add_features(prepared_df)

,bidPrice,askPrice,lastSalePrice,prev_close,last,spd_pct,pct_chg
symbol,,,,,,,
PBA,38.16,38.18,38.185,37.86,38.185,0.052411,0.858426
PGC,32.46,32.60,32.470,32.70,32.470,0.431300,-0.703364
URA,19.59,23.26,21.530,21.62,21.530,18.734048,-0.416281
AMWD,45.80,54.41,49.840,50.17,49.840,18.799127,-0.657764
CHKP,115.23,135.64,124.995,124.63,124.995,17.712401,0.292867
...,...,...,...,...,...,...,...
OVID,2.01,2.19,2.000,2.06,2.000,8.955224,-2.912621
LGF.B,8.35,9.08,8.360,8.30,8.360,8.742515,0.722892
SARK,51.12,55.76,55.750,54.76,55.750,9.076682,1.807889


In [57]:
# Top losers
(
    add_features(prepared_df)
    .sort_values(by='spd_pct').iloc[:2000]  # of the 2000 most liquid stocks
    .sort_values(by='pct_chg').head(10)  # display the top 10 losers
)

,bidPrice,askPrice,lastSalePrice,prev_close,last,spd_pct,pct_chg
symbol,,,,,,,
BCAC,10.17,10.31,10.230,17.600,10.230,1.376598,-41.875000
CORZ,2.21,2.24,2.235,2.510,2.235,1.357466,-10.956175
SNTI,2.31,2.34,2.330,2.610,2.330,1.298701,-10.727969
SOND,1.40,1.51,1.410,1.550,1.410,7.857143,-9.032258
EOSE,2.91,3.14,2.910,3.185,2.910,7.903780,-8.634223
VLD,2.93,2.95,2.935,3.210,2.935,0.682594,-8.566978
HIVE,4.09,4.11,4.120,4.500,4.120,0.488998,-8.444444
SUNW,2.20,2.21,2.200,2.380,2.200,0.454545,-7.563025
FTCI,4.73,4.76,4.725,5.090,4.725,0.634249,-7.170923


In [59]:
# Top gainers
(
    add_features(prepared_df)
    .sort_values(by='spd_pct').iloc[:2000]  # of the 2000 most liquid stocks
    .sort_values(by='pct_chg', ascending=False).head(10)  # display the top 10 gainers
)

,bidPrice,askPrice,lastSalePrice,prev_close,last,spd_pct,pct_chg
symbol,,,,,,,
SIGA,19.21,20.76,20.750,18.000,20.750,8.068714,15.277778
TNXP,1.91,1.93,1.920,1.675,1.920,1.047120,14.626866
AEMD,1.09,1.12,1.100,0.993,1.100,2.752294,10.775428
BTBT,1.47,1.49,1.480,1.375,1.480,1.360544,7.636364
CMRX,2.32,2.34,2.335,2.180,2.335,0.862069,7.110092
LGHL,1.03,1.05,1.050,0.995,1.050,1.941748,5.527638
GOVX,2.84,2.86,2.835,2.710,2.835,0.704225,4.612546
CAN,4.08,4.10,4.085,3.910,4.085,0.490196,4.475703
IGMS,16.80,16.94,16.890,16.180,16.890,0.833333,4.388133
